# ELAN to TEI conversion 

**Author:** Daniel Schopper    
**Description:** This notebook automates the ELAN to TEI conversion in the SHAWI Project. When finished, it should be ported to a non-interactive script which is triggered by a github action.    
**Last Change:** 2022-03-23     
**History:**    
* 2022-03-10: Initital set up
* 2022-03-11: added XSLT transformation via saxonpy
* 2022-03-12: added ELAN to TEI conversion
* 2022-03-23: integrated merge metadata XSL

## TODOS and Open Questions

* merge documents, making sure to not overwrite manual changes
* replace TEI documents with x-includes
* tokenize
* validate

In [1]:
import io
from azure.identity import ClientSecretCredential
import os
import logging
import requests
import pathlib
#import filetype – not used
from pathlib import Path
from urllib.parse import urlsplit
import saxonche
from zipfile import ZipFile
import subprocess
from datetime import datetime
# from inspect import getmembers, signature
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

## Configuration

In [2]:
# the URL of the Sharepoint installation 
# sp_baseURL = "oeawacat.sharepoint.com"

# the sharepoint username + password are taken from the environment
# sp_username = os.environ['SP_USERNAME']
# sp_pwd = pwd = os.environ['SP_PWD']
#sp_tenant_id = os.environ['SP_TENANT_ID']
#sp_client_id = os.environ['SP_CLIENT_ID']
#sp_client_secret = os.environ['SP_CLIENT_SECRET']
#sp_credentials = ClientSecretCredential(
#    tenant_id=sp_tenant_id,
#    client_id=sp_client_id,
#    client_secret=sp_client_secret
#)

# the name of the Sharepoint Site
sp_siteName = "ACDH-CH_p_ShawiTypeArabicDialects_Shawi"

# the path to the Excel file
# sp_pathToRecordingsXLSX = "Shared%20Documents/General/Shawi_Recordings.xlsx"
sp_RecordingsXSLX = "SHAWI_Recordings.xlsx" # is searched for recursively on SharePoint


# the name of the local directory where downloaded data will be stored
dataDir = "data"

# the name of the local directory where downloaded libraries and other auxiliary code will be stored
libDir = "lib"

# the root of the git repository
shawidataHomeDir = "../.."

# path to project-specific stylesheets
pathToShawiStylesheetsDir = shawidataHomeDir+"/082_scripts_xsl"

# the path to the ELAN transcription files
pathToELANDir = shawidataHomeDir+"/122_elan"

# the path to the non-annotated TEI transcription files
pathToTEIDir = shawidataHomeDir+"/103_tei_w"

# the path to the annotated TEI transcription files
pathToAnnotatedTEIDir = shawidataHomeDir+"/010_manannot"


# the path to the NoSkE verticals
noSkEVertDir = shawidataHomeDir+"/130_vert_plain"

# the path to the tei Corpus document produced by this script
pathToTeiCorpus = pathToTEIDir+"/shawiCorpus.xml"


# the path to the audio files
pathToRecordingsDir = "THIS_IS_NOT_USED"#"/mnt/univie_orientalistik/SHAWI/Recordings"

with saxonche.PySaxonProcessor(license=False) as proc:
    logging.info(proc.version)
    proc.set_cwd(os.path.dirname(os.path.abspath('')))
    logging.info(proc.cwd)


#set up directories
logging.info("** setting up directories **")
for i in [dataDir,libDir]: 
    if os.path.exists(i):
        logging.info("skipped existing directory '"+i+"'")
    else:
        os.mkdir(i)
        logging.info("created directory '"+i+"'")
        
        
# define which steps should be skipped. 

SKIP_PROCESSING = []#["runTEICorpo"]

2025-11-25 14:59:35,066 - SaxonC-HE 12.9 from Saxonica
2025-11-25 14:59:35,067 - Q:\basexshawi\shawi-data\080_scripts_generic
2025-11-25 14:59:35,068 - ** setting up directories **
2025-11-25 14:59:35,068 - skipped existing directory 'data'
2025-11-25 14:59:35,069 - skipped existing directory 'lib'


## Setup

### Step 1: get the latest release of the TEI Stylesheets 

In [3]:
# Setup

# fetch the TEI Stylesheets    
def installFromGithub(libraryName):
    auth = {}
    if 'GITHUB_TOKEN' in os.environ:
        auth = {"Authorization": "Bearer "+os.environ['GITHUB_TOKEN']}
    headers = {"Accept" : "application/vnd.github.v3+json"}
    repo = libraryName
    logging.info("** Fetching library "+repo+" **")
    libBasePath = libDir+"/"+repo
    
    # First we check which tag name the latest release has
    r = requests.get("https://api.github.com/repos/"+repo+"/releases/latest", headers={**headers, **auth})
    if r.status_code != 200:
        logging.error("An error occured fetching the latest release. Maybe there isn't any release? ")
        logging.error(r.content)
        return 1
    release = r.json()
    tag = release["tag_name"]
    
    # we check whether we have the latest version already \
    # by checking if the respective path is already installed
    libReleasePath = libBasePath+"/"+tag
    haveLatestVersion = os.path.exists(libReleasePath)
    if haveLatestVersion:
        logging.info("We have already the latest version ("+tag+"). Exiting")
        logging.info("")
        return libReleasePath
    else:
        url = release["assets"][0]["browser_download_url"]
        payload = requests.get(url, headers=auth).content
        zipfilename = os.path.basename(url)
        os.makedirs(libReleasePath, exist_ok=True)
        zipfilePath = libReleasePath +"/"+zipfilename
        open(zipfilePath, 'wb').write(payload)
        ZipFile(zipfilePath).extractall(path=libReleasePath)
        logging.info("Downloaded latest version ("+tag+") to "+libReleasePath)
        logging.info("")
        return libReleasePath


pathToTEIGuidelines=installFromGithub("TEIC/TEI")
pathToTEIStylesheets=installFromGithub("TEIC/Stylesheets")


2025-11-25 14:59:36,266 - ** Fetching library TEIC/TEI **
2025-11-25 14:59:37,024 - We have already the latest version (P5_Release_4.10.2). Exiting
2025-11-25 14:59:37,025 - 
2025-11-25 14:59:37,026 - ** Fetching library TEIC/Stylesheets **
2025-11-25 14:59:37,718 - We have already the latest version (v7.59.0). Exiting
2025-11-25 14:59:37,718 - 


### Step 2: Download the latest version of the Excel Sheet

source https://medium.com/@g3.prashanna/automating-sharepoint-file-downloads-in-pyspark-with-microsoft-graph-api-b9860b18cc2f
modified for using site name and team name instead of site id
The msgraph python class provided by microsoft lacks the ability to get acces by site name and team name and is a bit hard to use.
Check the Graph API documentation for more details: https://learn.microsoft.com/en-us/graph/api/resources/sharepoint?view=graph-rest-1.0
and future improvements of the msgraph python sdk.

This class provides functionality to download files from a SharePoint site using the Microsoft Graph API.
It authenticates using Azure AD credentials and allows file retrieval by specifying the site and team names,
rather than the site ID. It handles authentication, site and drive lookup,
recursive file search, and file download.
Usage Example:
```python
    downloader = SharePointDownloader(credentials, 'oeawacat.sharepoint.com', 'ACDH-CH_p_ShawiTypeArabicDialects_Shawi')
    with open("SHAWI_Recordings.xlsx", "wb") as f:
        f.write(downloader.download_file('SHAWI_Recordings.xlsx'))
```

In [4]:
class SharePointDownloader:
    def __init__(self, credentials, site_name, team_name):
        self.client_credential = credentials
        site_id = self._get_site_id_by_name(site_name, team_name)
        self.site_id = site_id

    def _get_token(self):
        return self.client_credential.get_token(
            "https://graph.microsoft.com/.default"
        ).token

    def _get_graph_headers(self):
        token = self._get_token()
        return {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

    def _get_site_id_by_name(self, site_name, team_name):
        url = f"https://graph.microsoft.com/v1.0/sites/{site_name}:/sites/{team_name}"
        response = requests.get(url, headers=self._get_graph_headers())
        response.raise_for_status()
        return response.json().get("id")

    def _get_drive_id(self):
        url = f"https://graph.microsoft.com/v1.0/sites/{self.site_id}/drive"
        response = requests.get(url, headers=self._get_graph_headers())
        response.raise_for_status()
        return response.json().get("id")

    def _list_content(self, headers, drive_id, item_id="root", source_file_name=None):
        items_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/children"
        response = requests.get(items_url, headers=headers)
        response.raise_for_status()
        items = response.json().get("value", [])
        
        for item in items:
            if "folder" in item:
                result = self._list_content(headers, drive_id, item["id"], source_file_name)
                if result:
                    return result
            if item["name"] == source_file_name:
                return item
        return None
    
    def download_file(self, file_name):
        drive_id = self._get_drive_id()
        headers = self._get_graph_headers()
        file_item = self._list_content(headers, drive_id, source_file_name=file_name)
        
        if not file_item:
            raise FileNotFoundError(f"{file_name} not found.")

        file_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{file_item['id']}/content"
        response = requests.get(file_url, headers=headers)
        response.raise_for_status()
        return response.content


pathToExcelSheet = "data/SHAWI_Recordings.xlsx"
#downloader = SharePointDownloader(sp_credentials, 'oeawacat.sharepoint.com', 'ACDH-CH_p_ShawiTypeArabicDialects_Shawi')
#with open(pathToExcelSheet, "wb") as f:
#    f.write(downloader.download_file(sp_RecordingsXSLX))
logging.info(pathToExcelSheet)

2025-11-25 14:59:39,428 - data/SHAWI_Recordings.xlsx


## Step 2: transform xlsx to TEI table

In [5]:
def transform(s, xsl, o, parameters=[]):
    # processor keeps files open on Windows and in doing so prevents moving or copying them
    with saxonche.PySaxonProcessor(license=False) as proc:
        proc.set_configuration_property("xi", "on")
        saxon = proc.new_xslt30_processor()
        for i in parameters:
            saxon.set_parameter(name=i, value=proc.make_string_value(parameters[i]))
        try:
            exec = saxon.compile_stylesheet(stylesheet_file=os.path.abspath(xsl))
            exec.set_global_context_item(file_name=os.path.abspath(s))
            # From the docs saxonc.html#PyXsltExecutable-set_initial_match_selection
            # This method does not set the global context item for the transformation;
            # if that is required, it can be done separately using the set_global_context_item method.
            exec.apply_templates_returning_file(source_file=os.path.abspath(s), output_file=os.path.abspath(o))
        except saxonche.PySaxonApiError as e:
            logging.info(str(e))
            logging.info(os.path.abspath(s)+" - "+os.path.abspath(xsl)+" -> "+os.path.abspath(o)+" failed")
        if proc.exception_occurred:
            logging.info(proc.get_error_message())
            logging.info(os.path.abspath(s)+" - "+os.path.abspath(xsl)+" -> "+os.path.abspath(o)+" failed")
        if os.path.exists(os.path.abspath(o)):
            return o
        else: 
            logging.info("there was an error transforming "+s+" with stylesheet "+xsl)

In [6]:
def xlsx2teitable(xlsx, output):

    # first, extract contents of XLSX document to a temp directory
    unzipPath=xlsx.replace(".xlsx","")
    os.makedirs(unzipPath, exist_ok=True)
    ZipFile(xlsx).extractall(path=unzipPath)
    
    # then transform the .rels file using the TEIC Stylesheets 
    pathToXlsxtoteiXSL=pathToTEIStylesheets+"/xml/tei/stylesheet/xlsx/xlsxtotei.xsl"

    params = {
        "inputDir" : pathlib.Path(os.path.abspath(unzipPath)).as_uri(),
        "workDir" : pathlib.Path(os.path.abspath(unzipPath)).as_uri()
        
    }

    transform(
        s = unzipPath+"/_rels/.rels", 
        xsl = pathToXlsxtoteiXSL, 
        o = output, 
        parameters=params
    )
    return

In [7]:
pathToTEItable=pathToExcelSheet.replace(".xlsx",".xml")

if not "xlsx2teitable" in SKIP_PROCESSING:    
    xlsx2teitable(xlsx=pathToExcelSheet, output=pathToTEItable)
    debugstring="""<!-- 
   THIS FILE IS INCLUDED IN THE GIT REPOSITORY ONLY FOR DEBUGGING PURPOSES. 
   
   The source of this file is constantly being edited at 
   https://oeawacat.sharepoint.com/sites/ACDH-CH_p_ShawiTypeArabicDialects_Shawi/_layouts/15/Doc.aspx?sourcedoc={F01FF43B-2409-4E31-A5BF-653E0559B160}&file=SHAWI%20Recordings.xlsx&action=default&mobileredirect=true&cid=f7311564-c2b6-4b08-9a52-468547688408
   So this copy is most probably already outdated.
   
  To update it, you can either run https://gitlab.com/acdh-oeaw/shawibarab/shawi-data/-/blob/main/080_scripts_generic/080_01_ELAN2TEI/ELAN2TEI.ipyn
   *OR*  
   1) download the Excel file manually from Sharepoint
   2) and tranform it to TEI using oxgarage.tei-c.org/ 
   
-->
    """
    f = open(pathToTEItable,mode="r",encoding="UTF8")
    src = f.read()
    new = src.replace('<?xml version="1.0" encoding="UTF-8"?>','<?xml version="1.0" encoding="UTF-8"?>\n'+debugstring)
    f.close()
    f = open(pathToTEItable, mode="wt",encoding="UTF8")
    f.write(new)
    f.close()
        
    logging.info(pathToTEItable)

2025-11-25 14:59:43,029 - data/SHAWI_Recordings.xml


## Step 3: transform TEI table to corpus header

In [8]:
pathToTeitableToCorpusXSL=pathToShawiStylesheetsDir+"/table2corpus.xsl"
pathToMergeMannannotHeaders=pathToShawiStylesheetsDir+"/mergeManannotHeaders.xsl"
params = {
    "pathToRecordings" : pathlib.Path(os.path.abspath(pathToRecordingsDir)).as_uri()
}
try:
    transform(pathToTEItable, pathToTeitableToCorpusXSL, pathToTeiCorpus, params)
    transform(pathToTeiCorpus, pathToMergeMannannotHeaders, pathToTeiCorpus+"-merged.xml")
    os.replace(pathToTeiCorpus+"-merged.xml", pathToTeiCorpus)
except saxonche.PySaxonApiError as e:
    logging.error("an error occured: " + str(e) + "\n" + pathToTEItable + ": " + pathToTeitableToCorpusXSL + " -> " + pathToTeiCorpus)
logging.info(pathToTeiCorpus)

2025-11-25 14:59:49,377 - ../../103_tei_w/shawiCorpus.xml


## Step 4: Run TEICorpo

In [9]:
def installFromUrl(url, force=False):
    r = requests.get(url)
    filename = os.path.basename(urlsplit(url).path)
    downloadpath = libDir+"/"+filename
    if os.path.exists(downloadpath) and not force:
        logging.info("skipping download")
    else:
        open(downloadpath, 'wb').write(r.content)
        logging.info("file "+downloadpath+" downloaded")
    return downloadpath

# TODO check for filetype and automatically extract zip file 
# so this can be re-used for the insta
 
installFromUrl("https://github.com/christopheparisse/teicorpo/blob/689abf780eeb945a02f2c98c797af2417d562562/teicorpo.jar?raw=true")
installFromUrl("https://repo1.maven.org/maven2/commons-io/commons-io/2.11.0/commons-io-2.11.0.jar")
pathToTeiCorpo=libDir+"/*"
logging.info(pathToTeiCorpo)

2025-09-15 09:33:04,376 - skipping download
2025-09-15 09:33:04,805 - skipping download
2025-09-15 09:33:04,807 - lib/*


Collect all ELAN documents from pathToELANDir

In [10]:
ELANDocs = []

for i in os.scandir(pathToELANDir):
    filename=os.path.basename(i)
    if filename.endswith(".eaf"):
        basename=Path(i).stem
        ELANDocs.append({
            "filepath" : os.path.abspath(i),
            "filename" : filename,
            "basename" : basename
        })
        
        # check whether there is already a manually annotated TEI version of this ELAN document
        TEI_annotated_filename=pathToAnnotatedTEIDir+"/"+basename+".xml"
        
        ELANDocs[-1]["TEI_annotated"]=os.path.abspath(TEI_annotated_filename)
for d in ELANDocs:
    logging.info(d["filepath"])

2025-09-15 09:33:04,815 - C:\Users\mrauschsupola\Shawi\shawi-data\122_elan\Urfa-000_Cigkofte-Harran-2001.eaf
2025-09-15 09:33:04,816 - C:\Users\mrauschsupola\Shawi\shawi-data\122_elan\Urfa-002a_Joke_about_a_tribe.eaf
2025-09-15 09:33:04,817 - C:\Users\mrauschsupola\Shawi\shawi-data\122_elan\Urfa-002b_Blood_feud_in_the_past.eaf
2025-09-15 09:33:04,817 - C:\Users\mrauschsupola\Shawi\shawi-data\122_elan\Urfa-002c_The_incomplete_meal.eaf
2025-09-15 09:33:04,817 - C:\Users\mrauschsupola\Shawi\shawi-data\122_elan\Urfa-002d_The_bull_in_the_jar.eaf
2025-09-15 09:33:04,818 - C:\Users\mrauschsupola\Shawi\shawi-data\122_elan\Urfa-011_Cemetry-Harran-2010.eaf
2025-09-15 09:33:04,818 - C:\Users\mrauschsupola\Shawi\shawi-data\122_elan\Urfa-012_Lentils-Harran-2010.eaf
2025-09-15 09:33:04,818 - C:\Users\mrauschsupola\Shawi\shawi-data\122_elan\Urfa-014_Village_of_Qoran-Harran-2010.eaf
2025-09-15 09:33:04,820 - C:\Users\mrauschsupola\Shawi\shawi-data\122_elan\Urfa-024a_Vegetabels_and_Seasonal_Work-Harran

In [11]:
def runTEICorpo(docs = dict):
    runtime = datetime.now().strftime("%Y-%m-%d_%H-%M")
    tmpDir = pathToTEIDir+"/"+runtime
    os.makedirs(tmpDir, exist_ok=True)
    for i in docs:
        pathToInput = i["filepath"]
        filenameELAN = i["filename"]
        filenameTEI = i["basename"]+".xml"
        pathToOutput = tmpDir+"/"+"ELAN_"+filenameTEI
        i["filepath_tmp_TEI"] = os.path.abspath(pathToOutput)
        i["tmpDir"] = tmpDir
        output = os.path.abspath(pathToTEIDir + "/" + i["basename"] + ".xml")
        i["TEI"] = os.path.abspath(output)
        res = subprocess.run(["java", "-cp", pathToTeiCorpo, "-Dfile.encoding=UTF-8", "fr.ortolang.teicorpo.TeiCorpo", "-from","elan", "-to","tei", "-o",pathToOutput, pathToInput], capture_output=True, encoding="UTF-8")
        print(res.stdout)
        print(res.stderr)
        print(pathToOutput)

run TEI Corpo on all ELANDocs, writing the path to the TEI output back to the variable

In [12]:
if not "runTEICorpo" in SKIP_PROCESSING:
    runTEICorpo(docs=ELANDocs)

TeiCorpo (version 1.4.44) 14/12/2021 08:07 Version TEI_CORPO: 0.9.1


../../103_tei_w/2025-09-15_09-33/ELAN_Urfa-000_Cigkofte-Harran-2001.xml
TeiCorpo (version 1.4.44) 14/12/2021 08:07 Version TEI_CORPO: 0.9.1


../../103_tei_w/2025-09-15_09-33/ELAN_Urfa-002a_Joke_about_a_tribe.xml
TeiCorpo (version 1.4.44) 14/12/2021 08:07 Version TEI_CORPO: 0.9.1


../../103_tei_w/2025-09-15_09-33/ELAN_Urfa-002b_Blood_feud_in_the_past.xml
TeiCorpo (version 1.4.44) 14/12/2021 08:07 Version TEI_CORPO: 0.9.1


../../103_tei_w/2025-09-15_09-33/ELAN_Urfa-002c_The_incomplete_meal.xml
TeiCorpo (version 1.4.44) 14/12/2021 08:07 Version TEI_CORPO: 0.9.1


../../103_tei_w/2025-09-15_09-33/ELAN_Urfa-002d_The_bull_in_the_jar.xml
TeiCorpo (version 1.4.44) 14/12/2021 08:07 Version TEI_CORPO: 0.9.1


../../103_tei_w/2025-09-15_09-33/ELAN_Urfa-011_Cemetry-Harran-2010.xml
TeiCorpo (version 1.4.44) 14/12/2021 08:07 Version TEI_CORPO: 0.9.1


../../103_tei_w/2025-09-15_09-33/ELAN_Urfa-012_Lentils-Harran-2010.xml
TeiCor

## Step 5: Merge metadata and TEICorpo Output

In [13]:
def mergeMetadata(docInfo, p):
    return transform(
        s = docInfo["filepath_tmp_TEI"],
        xsl = pathToShawiStylesheetsDir+"/mergeHeaderAndTranscription.xsl",
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_00_metaMerged.xml",
        parameters = p)

## Step 6: Tokenization of unannotated texts

Run a local copy of [xsl-tokenizer](https://github.com/acdh-oeaw/xsl-tokenizer)

The merged TEI document is tokenized for further manual annotation.

### Step 6.0: (Re-)generate tokenizer stylesheets (optional)

Regenerate the XSLs used in the following steps.
This can not be done with saxonpy (xincludes are not resolved)
use
```bash
java -jar Saxon-HE-9.9.1-8.jar -s:profile.xml -xi:on -xsl:xsl/make_xsl.xsl
```

For all the ELAN files converted to TEI:

### Step 6.1: Remove new lines

Remove new lines and store to intermediate document:

In [14]:
def removeNL(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_00_metaMerged.xml", 
        xsl = "./tokenizer/xsl/rmNl.xsl", 
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_01_nlRmd.xml"
    )

### Step 6.2: create w tags


In [15]:
def tokenize(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_01_nlRmd.xml", 
        xsl = "./tokenizer/wrapper_toks.xsl", 
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_02_toks.xml"
    )

### Step 6.3: Add part attributes to w tags

Add Part-Attributes and explicit token links:

In [16]:
def addP(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_02_toks.xml", 
        xsl = "./tokenizer/wrapper_addP.xsl", 
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_03_tokenized.xml"
    )

### Step 6.4: apply project-specific post-processing

Do some post tokenization processing specific to the Shawi project.

In [17]:
def postProcess(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_03_tokenized.xml", 
        xsl = "./tokenizer/postTokenization/1.xsl", 
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_04_posttok.xml"
    )

### Step 6.5: change annotationBlocks into divs and move media elements into one place

In [18]:
def annotationBlock2div(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_04_posttok.xml",
        xsl = "./tokenizer/annotationBlock2div.xsl",
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_05_ab2div.xml"
    )

### Step 6.6: move token namespace from xtoks to TEI 

**--> This step creates the files which data curators will copy to `010_manannot` and annotate using the TEI enricher**

In [19]:
def createTEIForAnnotation(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_05_ab2div.xml",
        xsl = "./tokenizer/custom_xtoks2tei.xsl", 
        o = docInfo["TEI"],
        parameters = {
            "preserve-ws": "false"
        }
    )

## Step 7: Create NoSke input

We create verticals from the unannotated texts and attach the token annotations from `010_manannot` to them.

### Step 7.1 Create XML vertical from tokenized XML documents

We take the tokenized XML document (prior to have moved to TEI) and create an XML vertical from it:

In [20]:
def createXMLVert(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_04_posttok.xml",
        xsl = "./tokenizer/custom_xtoks2vert.xsl", 
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_05_vert.xml"
    )

### Step 7.2: attach manual annotations to the XML vertical

In [21]:
def attachAnnotationsToXMLVert(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_05_vert.xml",
        xsl = pathToShawiStylesheetsDir+"/copyAnaToVert.xsl", 
        o = docInfo["tmpDir"] + "/" + docInfo["basename"] + "_05_vert_annot.xml",
        parameters = {
            "path_to_annotated_doc": pathlib.Path(os.path.abspath(docInfo["TEI_annotated"])).as_uri()
        }
    )

### Step 7.3 convert XML vertical to text vertical

Create a vertical vor NoSkE

In [22]:
def createNoSkEVert(docInfo):
    return transform(
        s = docInfo["tmpDir"] + "/" + docInfo["basename"] + "_05_vert_annot.xml",
        xsl = "./tokenizer/wrapper_vert2txt.xsl", 
        o = noSkEVertDir + "/" + docInfo["basename"] + ".txt"
    )

## Run Steps 6- 8

In [ ]:
print(os.path.abspath(pathToTeiCorpus))
mergeParam = { "pathToCorpusDoc": pathlib.Path(os.path.abspath(pathToTeiCorpus)).as_uri() }
for doc in ELANDocs:
    print(doc["basename"]+': '+doc["filepath_tmp_TEI"]+" -> "+doc["TEI"])
    mergeMetadata(doc, mergeParam)
    removeNL(doc)
    tokenize(doc)
    addP(doc)
    postProcess(doc)
    annotationBlock2div(doc)
    createTEIForAnnotation(doc)
    #createXMLVert(doc)
    #attachAnnotationsToXMLVert(doc)
    #createNoSkEVert(doc)
    print(doc["basename"]+": done.")

C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\shawiCorpus.xml
Urfa-000_Cigkofte-Harran-2001: C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\ELAN_Urfa-000_Cigkofte-Harran-2001.xml -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\Urfa-000_Cigkofte-Harran-2001.xml


2025-09-15 09:33:41,107 - XTSE0020: Serialization parameter Q{.../saxon.sf.net/}line-length is not available in Saxon-HE

2025-09-15 09:33:41,108 - C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-000_Cigkofte-Harran-2001_04_posttok.xml - C:\Users\mrauschsupola\Shawi\shawi-data\080_scripts_generic\080_01_ELAN2TEI\tokenizer\annotationBlock2div.xsl -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-000_Cigkofte-Harran-2001_05_ab2div.xml failed
2025-09-15 09:33:41,108 - there was an error transforming ../../103_tei_w/2025-09-15_09-33/Urfa-000_Cigkofte-Harran-2001_04_posttok.xml with stylesheet ./tokenizer/annotationBlock2div.xsl
2025-09-15 09:33:41,112 - Unable to resolve <C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-000_Cigkofte-Harran-2001_05_ab2div.xml> into a Source. Line number: -1
2025-09-15 09:33:41,113 - C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-000_Cigkofte-Harran-2001_05_ab2di

Urfa-000_Cigkofte-Harran-2001: done.
Urfa-002a_Joke_about_a_tribe: C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\ELAN_Urfa-002a_Joke_about_a_tribe.xml -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\Urfa-002a_Joke_about_a_tribe.xml


2025-09-15 09:33:41,341 - XTMM9000: Processing terminated by xsl:message at line 34 in mergeHeaderAndTranscription.xsl. Line number: 34
2025-09-15 09:33:41,341 - C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\ELAN_Urfa-002a_Joke_about_a_tribe.xml - C:\Users\mrauschsupola\Shawi\shawi-data\082_scripts_xsl\mergeHeaderAndTranscription.xsl -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-002a_Joke_about_a_tribe_00_metaMerged.xml failed
2025-09-15 09:33:41,347 - SXXP0003   Error reported by XML parser: Premature end of file.. Line number: -1
2025-09-15 09:33:41,348 - C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-002a_Joke_about_a_tribe_00_metaMerged.xml - C:\Users\mrauschsupola\Shawi\shawi-data\080_scripts_generic\080_01_ELAN2TEI\tokenizer\xsl\rmNl.xsl -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-002a_Joke_about_a_tribe_01_nlRmd.xml failed
2025-09-15 09:33:41,349 - there was an error transfo

Urfa-002a_Joke_about_a_tribe: done.
Urfa-002b_Blood_feud_in_the_past: C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\ELAN_Urfa-002b_Blood_feud_in_the_past.xml -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\Urfa-002b_Blood_feud_in_the_past.xml
Urfa-002b_Blood_feud_in_the_past: done.
Urfa-002c_The_incomplete_meal: C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\ELAN_Urfa-002c_The_incomplete_meal.xml -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\Urfa-002c_The_incomplete_meal.xml


...

2025-09-15 09:34:32,816 - XTSE0020: Serialization parameter Q{.../saxon.sf.net/}line-length is not available in Saxon-HE

2025-09-15 09:34:32,816 - C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-202_Al-Shukwa-Harran-2017_04_posttok.xml - C:\Users\mrauschsupola\Shawi\shawi-data\080_scripts_generic\080_01_ELAN2TEI\tokenizer\annotationBlock2div.xsl -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-202_Al-Shukwa-Harran-2017_05_ab2div.xml failed
2025-09-15 09:34:32,818 - there was an error transforming ../../103_tei_w/2025-09-15_09-33/Urfa-202_Al-Shukwa-Harran-2017_04_posttok.xml with stylesheet ./tokenizer/annotationBlock2div.xsl
2025-09-15 09:34:32,820 - Unable to resolve <C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-202_Al-Shukwa-Harran-2017_05_ab2div.xml> into a Source. Line number: -1
2025-09-15 09:34:32,820 - C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-202_Al-Shukwa-Harran-2017_05_

Urfa-202_Al-Shukwa-Harran-2017: done.
Urfa-203_Tattooing-Harran-2017: C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\ELAN_Urfa-203_Tattooing-Harran-2017.xml -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\Urfa-203_Tattooing-Harran-2017.xml


2025-09-15 09:34:33,186 - XTSE0020: Serialization parameter Q{.../saxon.sf.net/}line-length is not available in Saxon-HE

2025-09-15 09:34:33,198 - C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-203_Tattooing-Harran-2017_04_posttok.xml - C:\Users\mrauschsupola\Shawi\shawi-data\080_scripts_generic\080_01_ELAN2TEI\tokenizer\annotationBlock2div.xsl -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-203_Tattooing-Harran-2017_05_ab2div.xml failed
2025-09-15 09:34:33,198 - there was an error transforming ../../103_tei_w/2025-09-15_09-33/Urfa-203_Tattooing-Harran-2017_04_posttok.xml with stylesheet ./tokenizer/annotationBlock2div.xsl
2025-09-15 09:34:33,201 - Unable to resolve <C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-203_Tattooing-Harran-2017_05_ab2div.xml> into a Source. Line number: -1
2025-09-15 09:34:33,201 - C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-203_Tattooing-Harran-2017_05_

Urfa-203_Tattooing-Harran-2017: done.
Urfa-352_Washing_Clothes-Harran-2021: C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\ELAN_Urfa-352_Washing_Clothes-Harran-2021.xml -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\Urfa-352_Washing_Clothes-Harran-2021.xml


2025-09-15 09:34:33,685 - XTSE0020: Serialization parameter Q{.../saxon.sf.net/}line-length is not available in Saxon-HE

2025-09-15 09:34:33,701 - C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-352_Washing_Clothes-Harran-2021_04_posttok.xml - C:\Users\mrauschsupola\Shawi\shawi-data\080_scripts_generic\080_01_ELAN2TEI\tokenizer\annotationBlock2div.xsl -> C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-352_Washing_Clothes-Harran-2021_05_ab2div.xml failed
2025-09-15 09:34:33,701 - there was an error transforming ../../103_tei_w/2025-09-15_09-33/Urfa-352_Washing_Clothes-Harran-2021_04_posttok.xml with stylesheet ./tokenizer/annotationBlock2div.xsl
2025-09-15 09:34:33,701 - Unable to resolve <C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-352_Washing_Clothes-Harran-2021_05_ab2div.xml> into a Source. Line number: -1
2025-09-15 09:34:33,701 - C:\Users\mrauschsupola\Shawi\shawi-data\103_tei_w\2025-09-15_09-33\Urfa-352_W

## Replace TEI elements with x-includes in corpus document